<a href="https://colab.research.google.com/github/sharmer156/rnn_lyrics/blob/master/test/RNN-modelin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import tensorflow as tf  #这里用的tf2
import os

In [9]:
read_file = '/content/drive/MyDrive/date-rnn/j.txt'  #你的文件位置
text = open(read_file, 'rb').read().decode(encoding='utf-8')

#清理文本中的特殊符号
def clean_text(text): 
  cleaned = text.strip().replace(' ','').replace('\u3000','').replace('\ufeff','').replace('(','').replace(')','')
  cleaned = cleaned.replace('\r', '')
  cleaned = cleaned.replace('：', '')
  return cleaned

after_clean = clean_text(text)
vocab = sorted(set(after_clean))  
# 整个文本有33042 characters, 
#不同字符的个数（词典大小）vocab size 2422

# char <-> idx
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in after_clean]) # shape(33042,)

In [11]:
seq_length = 20  # max input length 20
examples_per_epoch = len(after_clean)//seq_length   # 33042//20; 1652个句子, 如果改进的话， 分句再padd会更好
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):       # t时刻 挪到t+1
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '想要有直升机\n想要和你飞到宇宙去\n想要和'
Target data: '要有直升机\n想要和你飞到宇宙去\n想要和你'


In [14]:
BATCH_SIZE = 64       #  1652//64; 每个epoch 训练25次
BUFFER_SIZE = 2000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)    # embedding 参数
embedding_dim = 300
rnn_units = 1024
dataset

<BatchDataset shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           774300    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          4073472   
_________________________________________________________________
dense (Dense)                (64, None, 2581)          2645525   
Total params: 7,493,297
Trainable params: 7,493,297
Non-trainable params: 0
_________________________________________________________________


In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# 取一个训练数据查看一下
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss: ", example_batch_loss.numpy().mean())

(64, 20, 2581) # (batch_size, sequence_length, vocab_size)
Prediction shape:  (64, 20, 2581)  # (batch_size, sequence_length, vocab_size)
scalar_loss:  7.85585


In [9]:
# 检查点保存至的目录
checkpoint_dir = 'training_checkpoints'
# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [17]:
# 检查点保存至的目录
checkpoint_dir = '/content/drive/MyDrive/AImodel/training_checkpoints'
# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [10]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
44/44 [==============================] - 3s 27ms/step - loss: 7.3640
Epoch 2/50
44/44 [==============================] - 1s 24ms/step - loss: 6.0681
Epoch 3/50
44/44 [==============================] - 1s 24ms/step - loss: 5.6382
Epoch 4/50
44/44 [==============================] - 1s 24ms/step - loss: 5.1888
Epoch 5/50
44/44 [==============================] - 1s 24ms/step - loss: 4.7202
Epoch 6/50
44/44 [==============================] - 1s 24ms/step - loss: 4.2179
Epoch 7/50
44/44 [==============================] - 1s 24ms/step - loss: 3.7448
Epoch 8/50
44/44 [==============================] - 1s 24ms/step - loss: 3.2687
Epoch 9/50
44/44 [==============================] - 1s 24ms/step - loss: 2.8266
Epoch 10/50
44/44 [==============================] - 1s 24ms/step - loss: 2.4469
Epoch 11/50
44/44 [==============================] - 1s 25ms/step - loss: 2.0284
Epoch 12/50
44/44 [==============================] - 1s 25ms/step - loss: 1.7186
Epoch 13/50
44/44 [==================

In [18]:
# load weight to model for predict, reshape batch to 1
weight = tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(weight)
model.build(tf.TensorShape([1, None]))

In [19]:
def generate_text(model, start_string):
  
  # 要生成的字符个数
  num_generate = 290
  
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 创建存储结果的空列表
  text_generated = []

  temperature = 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)  # delete 1dim 

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
def generate_text(model, start_string):
  
  # 要生成的字符个数
  num_generate = 19
  
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 创建存储结果的空列表
  text_generated = []

  temperature = 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)  # delete 1dim 

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [20]:
print(generate_text(model, start_string=u'远方'))

远方我有爱才能够明白
走在淡水河衅
听着她的最爱
把温暖放回口袋
外婆她的无奈
无法变成期待
只有爱才能够明白
走在淡水河衅
听着她的最爱
把温暖放回口袋
外婆她的无奈远不用别人比较
只喝过冰过的饮料很少笑吃全麦的面包
是因为我太爱你你要我说多
难怪大人们说我吃你豆腐
千山万水无数黑夜等一轮明的让我感动的可爱女人
坏坏的让我疯狂的可能雨也不会停
黑色毛衣
藏在那里
就让回忆永远停在那里
看着远方别当路儿啦
啦儿啦啦儿啦啦儿啦儿啦
铜镜映无邪紮马尾
你若撒野今生我把酒奉陪
啦儿啦啦啦啦啦啦
天空是蒙蒙蒙的雾写完全失去之前
你说把爱渐渐
放下会走更远
又何必去改变
已走过的时间
你


In [21]:
print(generate_text(model, start_string=u'速度'))

速度地悔落
是谁流下不让同情发现
我对天空这个那个流星
大声唱
吵杂的情不再回
那些爱过的感觉都太深刻我都还记得
你不等了说好的幸福呢
我错了泪干了放手了后悔了
只是回忆的音乐盒还旋转着要怎么停呢
你的回想起你陪伴
而不是六百块
比你给的还简单
外婆她的无奈
无法变成期待
只有爱才能够明白
走在淡水河衅
听着她的最爱
把温暖放回口袋
外婆她的期待
慢慢变成无奈
大人们始终不明白
我站在滿心期待
读完成回不来
不要你离开回忆划不开
欠你的宠爱
还要听
那些爱过的时候你陪我
我不是孤獨的小丑你笑了之后不需要記得我。
燈熄的時候滿天的星空最明亮的是寂寞
对这些太威胁太强烈
从来不喊累


In [ ]:
def generate_text(model, start_string):
  
  # 要生成的字符个数
  num_generate = 100
  
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 创建存储结果的空列表
  text_generated = []

  temperature = 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)  # delete 1dim 

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u'烟雨'))

烟雨纷纷雨袅aYaHei
哦跑到哪儿去啦
面店旁的小水沟留着我的天气冷的让人生闷气
火气我沮丧的悲哀
只為永恒的乐曲存在醒过来
鐘逆时鐘而过滤了你和我沦落面成美
沉在盒子里的世界才能完美
他们猜随便猜不重
